# Google Speech Command Encoding with Mel-Spectrogram

In [ ]:
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import random
import pandas as pd
import re
from tqdm import trange
import copy
import hashlib

import scipy.io.wavfile
from scipy.fftpack import dct
import soundfile as sf

In [ ]:
# navigate to folder where http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz is downloaded
os.chdir("/its/home/ts468/data/GSC/speech_commands/") 

In [ ]:
# load GSC validation and training data list

# load a list of training audio files
validation_files = []
with open("validation_list.txt", "r") as file:
    for line in file:
        x = line[:-1]
        validation_files.append(x)
        
# load a list of testing audio files
test_files = []
with open("testing_list.txt", "r") as file:
    for line in file:
        x = line[:-1]
        test_files.append(x)

In [ ]:
# View sound file

print("file format: ", test_files[0])
ipd.Audio(test_files[0])

In [ ]:
# load classes and assign labels

folders = os.listdir()
exception = ["validation_list.txt",
             "_background_noise_",
             ".DS_Store",
             "README.md",
             "LICENSE",
             "testing_list.txt"]

assert len(list(set(folders) - set(exception))) == 35

classes = np.sort(list(set(folders) - set(exception)))

classes_and_labels = {}

for i, c in enumerate(classes):
    classes_and_labels[c] = i

In [ ]:
def to_mel_spectrogram(file_name, display = False, steps = 80):
    # load audio files with librosa
    signal, sr = librosa.load(file_name)

    # pre-emphasis filter
    alpha =  .95
    emphasized_signal = np.append(signal[0], signal[1:] - alpha * signal[:-1])
    
    # framing
    frame_size = .025
    frame_stride = .01

    frame_length, frame_step = frame_size * sr, frame_stride * sr  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    
    # Hamming window
    frames *= np.hamming(frame_length)
    
    # fast Fourier-Transform and Power Spectrum
    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
    
    # filter banks
    nfilt = 40
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (sr / 2) / 700))  # Convert Hz to Mel
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = np.floor((NFFT + 1) * hz_points / sr)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * np.log10(filter_banks)  # dB

    
    # Crop or pad to 80 steps by repeating the last frame
    target_steps = steps
    current_steps = filter_banks.shape[0]
    if current_steps < target_steps:
        padding = np.tile(filter_banks[-1, :], (target_steps - current_steps, 1))
        filter_banks = np.vstack((filter_banks, padding))
    elif current_steps > target_steps:
        filter_banks = filter_banks[:target_steps, :]
    
    # todo: this is a fix to allow output to match input...  
    filter_banks = np.rot90(filter_banks)
    filter_banks = np.flipud(filter_banks)
    
    if display:
        # Display the filter banks with the 'viridis' colormap
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(filter_banks, x_axis='time', y_axis='mel', sr=sr, cmap='viridis')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel Filter Banks with Pre-Emphasis Filter (Cropped/Padded to 80 Steps)')
        plt.tight_layout()
        plt.show()
        
        print(filter_banks.shape)
    
    else:
        return filter_banks

In [ ]:
# Visualise the output of the mel encoding

test_image = test_files[1230]
to_mel_spectrogram(test_image, True, 100)

# Visualise the same input but on soecgram (visual check)
data, samplerate = sf.read(test_image)  
Pxx, freqs, bins, im = plt.specgram(data, Fs=samplerate)
plt.show()

In [ ]:
# Get all files to loop through

all_files = []

for folder in list(set(folders) - set(exception)):
    files_in_folder = os.listdir(folder)
    for sound_file in files_in_folder:
        all_files.append(folder + "/" + sound_file)

In [ ]:
training_x_data = []
training_y_data = []
testing_x_data = []
testing_y_data = []
validation_x_data = []
validation_y_data = []

# save all to a list
for i in trange(len(all_files)):
    
    if all_files[i] in validation_files:
        validation_x_data.append(copy.deepcopy(to_mel_spectrogram(all_files[i])))
        validation_y_data.append(int(classes_and_labels[re.split("[/]", all_files[i])[0]]))

    if all_files[i] in test_files:
        testing_x_data.append(copy.deepcopy(to_mel_spectrogram(all_files[i])))
        testing_y_data.append(int(classes_and_labels[re.split("[/]", all_files[i])[0]]))
        
    else:
        training_x_data.append(copy.deepcopy(to_mel_spectrogram(all_files[i])))
        training_y_data.append(int(classes_and_labels[re.split("[/]", all_files[i])[0]]))

In [ ]:
print(len(training_x_data))
print(len(testing_x_data))
print(len(validation_x_data))

# Save dataset

# TODO: update

In [ ]:
# navigate to folder to save encoded data
os.chdir("/its/home/ts468/data/GSC/experimental_1")
print("current cwd", os.getcwd())

In [ ]:
np.save("training_x_data.npy", training_x_data)
np.save("training_y_data.npy", training_y_data)
np.save("testing_x_data.npy", testing_x_data)
np.save("testing_y_data.npy", testing_y_data)
np.save("validation_x_data.npy", validation_x_data)
np.save("validation_y_data.npy", validation_y_data)

In [ ]:
os.listdir()